# SMPC - Simple Aggregation & Multiplication Algorithms

This notebook serves as the initial integration of simple aggregation & multiplication algorithms between 2+ parties without revealing their secret values using open source protocols. This is achieved by splitting each value into multiple shares, each of which operate like a private key.

Important notes:
* each secret is split into 3 shares as default
* additive sharing, which needs all parties to participate, no drop outs (for such scenarios you need shamir's sharing)
* honest-but-curious (or passive) security

Implemented primitives:
* addition
* substraction
* comparison
* multiplication


## Preparations

In [1]:
# import modules
import random as rd  # generating random numbers for distributions
import numpy as np  # handling vectors and matrices
from generate_prime import generate_prime_number  # code for generating large prime numbers quickly

## Encryption

> Encryption doesn't use floats or real numbers but happens in a mathematical space called [integer quotient ring](http://mathworld.wolfram.com/QuotientRing.html) which is basically the integers between `0` and `Q-1`, where `Q` is prime and \"big enough\" so that the space can contain all the numbers that we use in our algorithms. In practice, given a value `x` integer, we do `x % Q` to fit in the ring. (That's why we avoid using number `x' > Q`).

In [2]:
Q = generate_prime_number(50)
Q

878453306505433

In [3]:
# enryption function (additive sharing)
def encrypt(x, n_shares=3):
    shares = tuple(rd.randrange(0,Q) for _ in range(n_shares-1))
    return (*shares, ((x - sum(shares)) % Q))

## Decryption

In [4]:
# decryption function
def decrypt(*shares):
    return sum(shares) % Q

In [5]:
# example encryption
ex_shares = encrypt(6041)
ex_shares

(166161326824824, 134444947934158, 577847031752492)

In [6]:
decrypt(*ex_shares)

6041

## Aggregation Functions (`+` and `-`)

`x + y = (x0 + x1 + x2) + (y0 + y1 + y2)`

`x + y = (x0 + y0) + (x1 + y1) + (x2 + y2)`

In [7]:
# addition algorithm
def add(x, y):
    # x & y have to have the same number of shares (= length)
    return [(xi + yi) % Q for xi, yi in zip(x,y)]

In [8]:
# substraction algorithm
def sub(x, y):
    # x & y have to have the same number of shares (= length)
    return [(xi - yi) % Q for xi, yi in zip(x,y)]

### Example for adding 3 secrets

In [9]:
# example
# set 3 secret values and encrypt each into 3 (default) shares
x = encrypt(6041)
y = encrypt(59)
z = encrypt(900)
# what do the shares look like
[x, y, z]

[(240878361736014, 727039454958165, 788988796322728),
 (552308905462025, 285508428374134, 40635972669333),
 (255390414474448, 765759442661277, 735756755876041)]

In [10]:
# add the encrypted values
agg = add(add(x,y),z)
agg

[170124375167054, 21400712982710, 686928218362669]

In [11]:
# and decrypt the aggregated shares
decrypt(*agg)

7000

### Now substracting x and z

In [12]:
decrypt(*sub(x,z))

5141

## Now checking for equality

In [13]:
x2 = encrypt(6041)

In [14]:
x==x2

False

In [15]:
decrypt(*sub(x,x2))

0

## Multiplication Function

General:

* Using Beaver's precomputed multiplication triples

---

`x * y = (x0 + x1) * (y0 + y1)`

`x * y = (x0 * y0) + (x1 * y1) + (x0 * y1) + (x1 * y0)`

---

Masking (Beaver's triple, can only be used for one calculation):
* masks: s, t
* masked values: alpha, beta
* Generated by third party during offline phase, because masks are independent of private values

---

Public Masked Values:

`alpha = (x0 - s0) + (x1 - s1)`

`beta = (y0 - t0) + (y1 - t1)`

---

`z0 = st0 + (s0 * beta) + (alpha * t0) + (alpha * beta)`

`z1 = st1 + (s1 * beta) + (alpha * t1)`

---

Equivalency:
`z0 + z1 = x * y` (while x and y where perfectly hidden during the calculation)

In [16]:
# generate additional independent masks which have a multiplicative relationship
def generate_mul_triple(n_shares=3):
    # create triple
    s = rd.randrange(0,Q)
    t = rd.randrange(0,Q)
    st = (s*t)%Q
    
    # create shares
    s = encrypt(s, n_shares=n_shares)
    t = encrypt(t, n_shares=n_shares)
    st = encrypt(st, n_shares=n_shares)
    
    return s,t,st

# multiplication algorithm
def mul(x,y):
    # get number of shares
    n_shares = len(x)
    # create triple from trusted party
    s,t,st = generate_mul_triple(n_shares=n_shares)
    
    # create one-time-pad encryptions (each party has a share of each)
    alpha = sum(sub(x,s))
    beta = sum(sub(y,t))
    
    # create z with additive relationship (using indicator function to only add alpha*beta once)
    z = [(st[i] + (alpha * t[i]) + (beta * s[i]) + int(i==0)*(alpha*beta)) % Q for i in range(n_shares)]
    
    return z

### Example for multiplying 2 secrets

In [17]:
x = encrypt(50, n_shares=5)
y = encrypt(12, n_shares=5)

In [18]:
z = mul(x,y)

In [19]:
z

[245154185535248,
 288884198386305,
 538147406504582,
 110792568711,
 684610030016620]

In [20]:
decrypt(*z)

600

## Creating Classes of Private and Public Values - WIP

In [21]:
class PublicValue:
    
    def __init__(self, value):
        self.value = value

In [22]:
class PrivateValue:

    def __init__(self, value, share0=None, share1=None, share2=None):
        if not value is None:
            share0, share1, share2 = encrypt(value)
        self.share0 = share0
        self.share1 = share1
        self.share2 = share2

    def decrypt(self):
        return PublicValue(decrypt(self.share0, self.share1, self.share2))
    
    def add(x, y):
        if type(y) is int: y = PublicValue(y)
        if type(y) is PublicValue:
            share0 = (x.share0 + y.element) % Q
            share1 =  x.share1
            share2 =  x.share2
            return PrivateValue(None, share0, share1, share2)
        if type(y) is PrivateValue:
            share0 = (x.share0 + y.share0) % Q
            share1 = (x.share1 + y.share1) % Q
            share2 = (x.share2 + y.share2) % Q
            return PrivateValue(None, share0, share1, share2)

In [23]:
x = PrivateValue(5)
y = PrivateValue(12)

In [24]:
z = x.add(y=y)

In [25]:
decrypt(z.share0, z.share1, z.share2)

17